# Introduction to Functional Programming
Bianca Gibson

# What should you get out of this?

- What is the core of FP
- How can it help with problems you've experienced?
- How to use it in projects you're already working on
- Not necessarily hard and complicated
- No neckbeard required

# What we will cover

- What makes a functional language?
- Immutability
- Putting off side effects
- Referential transparency
- Common traps when explaining to others
- How to learn more

# Stop things changing behind my back

# What makes a functional language?

- First class functions - can pass functions to other functions
- other stuff is nice (which python has some of!) but that's the basics to make it workable
- don't need a 'functional' language - I'm a ruby dev and I use mutation so rarely it attracts comments if I do

# Immutability

- Don't change values, use constants and the stack
- Instead of changing, create a new one
- Recursion and accumulators over loops and mutation
- how many of you are familiar with recursion?
- Confusion over passing by value or reference? Gone

# Story

My gradle dependency list mutability story

# Code examples

In [4]:
class Item:
    def __init__(self, cost):
        self.cost = cost
        
    def __eq__(self, other):
        if type(other) is type(self):
            return self.cost == other.cost
        return false
    
    def __ne__(self, other):
        return not self.__eq__(other)

In [5]:
GST_MULTIPLIER = 1.1

def with_gst(item):
    return Item(item.cost * GST_MULTIPLIER)

def add_gst_explicit_recursion(items):
    head, *tail = items
    if len(tail) == 0:
        return (with_gst(head),)
    else:
        return (with_gst(head),) + add_gst_explicit_recursion(tail)

print(add_gst_explicit_recursion([Item(10), Item(5)]))

(<__main__.Item object at 0x7feb6d2ac518>, <__main__.Item object at 0x7feb6d2ac550>)


In [6]:
def add_gst(items):
    return [with_gst(i) for i in items]

add_gst([Item(10), Item(5)])

[<__main__.Item at 0x7feb6d2acd30>, <__main__.Item at 0x7feb6d2acdd8>]

In [7]:
def map_add_gst(items):
    return map(with_gst, items)

map_add_gst([Item(10), Item(5)])

In [8]:
# iterative and mutable

def mutable_total_cart(items):
    total = 0
    for item in items:
        total += item.cost
    return total

mutable_total_cart([Item(10), Item(5)])

15

In [9]:
# explicit recursion

def recursive_total_cart(items):
    head, *tail = items
    if len(tail) == 0:
        return head.cost
    else:
        return head.cost + recursive_total_cart(tail)

recursive_total_cart([Item(10), Item(5)])

15

In [10]:
# tail call recursion

def tail_call_total_cart(items, total=0):
    head, *tail = items
    if len(tail) == 0:
        return total + head.cost
    else:
        return tail_call_total_cart(tail, total + head.cost)
    
tail_call_total_cart([Item(10), Item(5)])

15

In [11]:
#implicit recursion

from functools import reduce

def total_cart(items):
    return reduce(lambda total, head: total + head.cost, items, 0)
    
total_cart([Item(10), Item(5)])

15

# What are side effects?

- when your code effects the outside world
- Printing
- writing to disk
- network calls
- it's why you're writing software, what it does

# What do they have to do with FP?

- separate logic from side effects
- push them to the edge of your system
- in the middle make a representation
- make core easier to test and reason about
- no side effects -> simpler tests
- Push hard testing out to the edge, isolated
- not about getting rid of them, about controlling them

In [12]:
import sys

def print_add_gst(items, out=sys.stdout):
    [out.write("%s\n" % (str(with_gst(i).cost))) for i in items]

print_add_gst([Item(10), Item(5)])

11.0
5.5


In [13]:
def print_cost(items):
    [sys.stdout.write("%s\n" % (str(i.cost))) for i in items]

print_cost(add_gst([Item(10), Item(5)]))

11.0
5.5


In [14]:
import unittest
from io import StringIO

def test_print_add_gst():
    out = StringIO()
    print_add_gst([Item(10), Item(5)], out=out)
    assert out.getvalue().strip() == "11.0\n5.5"
    
test_print_add_gst()

In [15]:
import unittest

def test_add_gst():
    assert add_gst([Item(10), Item(5)]) == [Item(11.0), Item(5.5)]
    
test_add_gst()

# Referential transparency

- combination of immutability and no side effects
- call a function again, get the same result and no side effects
- can substitute saved result for function call
- everything a function does represented by the result -> easy reasoning, nothing unexpected

In [16]:
# referentially transparent

def add_gst_cost(item):
    return with_gst(item)

item = Item(10)
print(add_gst_cost(item).cost)
print(add_gst_cost(item).cost)

11.0
11.0


In [17]:
# not referentially transparent

def mutable_add_gst_cost(item):
    item.cost = with_gst(item).cost
    return item

item = Item(10)
print(mutable_add_gst_cost(item).cost)
print(mutable_add_gst_cost(item).cost)

11.0
12.100000000000001


# Tail call optimisation

- Makes FP much more efficient by not creating a new stack from for every tail call
- Not in python, not going to happen
- Guido's reasons were a bit off, and when pressed enough he basically said "I don't like it"
- Use HO functions - functions that take functions - like map and filter instead
- Also more concise and clearer once you've got your eye in - doesn't have the recursion boilerplate
- They are actually iterative and mutating in python, but you can reason about it as pure
- also trampolines available if you want explicit recursion

# Clash with OO?

- Some claim that they clash
- Say that mutation of state is fundamental to OO
- Just making new objects instead of changing them has nothing to do with OO
- Command query separation - related to putting off side effects

# Teaching

- don't get too mathematical, since FP has that reputation
- relate to stuff they know - like monads they have already used
- explaining monads to Venetia
- avoid words like monad until they understand it
- focus on advantages to them, fight the academic and impractical image

# Learning more

Got a link to slides at the end, it's ok if you don't catch the link

# docs.python.org/3/howto/functional.html

- Useful examples
- advocates for local mutation, claims that OO and FP clash

# ibm.com/developerworks/linux/library/l-prog/index.html

# TCO implementation:
## baruchel.github.io/python/2015/11/07/explaining-functional-aspects-in-python/

# bogotobogo.com/python/python_fncs_map_filter_reduce.php

# diveintopython.net/functional_programming/index.html

# anandology.com/python-practice-book/functional-programming.html

# Functional Programming in Python, David Mertz

# Non Python resources

# Functional Programming Principles in Scala - coursera

# Functional Programming in Scala - Paul Chiusano & Rúnar Bjarnason

# Refactoring Ruby with Monads by Tom Stuart

# github.com/cwmyers/FunctionalTraining

# github.com/NICTA/course

# Stop things changing behind our backs

# Questions
## github.com/biancaG/fp-for-pythonistas/tree/compcon
## bianca.rachel.gibson@gmail.com